### installs

In [1]:
%pip install imbalanced-learn
%pip install flaml



[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### imports

In [2]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from flaml import AutoML
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### companies

In [ ]:
companies = [
    "poco f2",
    "iphone",
    "galaxy",
    "uplift",
    "iphone (apple)",
    "realme",
    "xgody",
    "xiamoi",
    "huawei",
    "mediatek",
    "asus",
    "lg",
    "zte",
    "tecno",
    "motorola",
    "poco m2",
    "vivo",
    "moto",
    "itel",
    "apple",
    "acer",
    "dimensity",
    "poco",
    "redmi",
    "hmd",
    "samsung",
    "hmd global (nokia)",
    "micromax",
    "xiaomi",
    "ismart",
    "galaxy (samsung)",
    "one plus",
    "black shark (xiaomi )",
    "tecno",
    "lava",
    "qualcomm",
    "koomus",
    "exynos",
    "iqoo",
    "i kall",
    "sony",
    "iphone(apple)",
    "bsnl",
    "jio",
    "galaxynote20ultra",
    "realmex7pro",
    "google",
    "coolpad",
    "aspera",
    "oppo",
    "honor",
    "oneplus",
    "nokia",
    "pixel",
    "infinix",
    "potronics",
    "mi",
    "axon",
    "lantronix",
    "pocox3",
    "blackberry",
    "पोको एफ 2",
    "आइफोन",
    "गैलेक्सी",
    "अपलिफ्ट",
    "आइफोन (एप्पल)",
    "रियलमी",
    "एक्सगोडी",
    "शाओमी",
    "हुवाई",
    "मीडियाटेक",
    "आसूस",
    "एलजी",
    "जेडटीई",
    "टेक्नो",
    "मोटोरोला",
    "पोको एम 2",
    "वीवो",
    "मोटो",
    "मी",
    "आईटेल",
    "एप्पल",
    "एसर",
    "पोको",
    "रेडमी",
    "एचएमडी",
    "सैमसंग",
    "एचएमडी ग्लोबल (नोकिया)",
    "माइक्रोमैक्स",
    "आइस्‍मार्ट",
    "गैलेक्सी (सैमसंग)",
    "वन प्लस",
    "ब्लैक शार्क (शाओमी)",
    "टेकोनो",
    "लावा",
    "क्वालकॉम",
    "कुओमस",
    "एक्सिनोस",
    "आई कॉल",
    "सोनी",
    "बीएसएनएल",
    "जियो",
    "गूगल",
    "कूलपैड",
    "ओप्पो",
    "हॉनर",
    "वनप्लस",
    "नोकिया",
    "पिक्सेल",
    "पोट्रोनिक्स",
    "मी",
    "एक्सोन",
    "लैनट्रोनिक्स"
]


### reading csv files

In [5]:
dfa = pd.read_csv(
    "https://raw.githubusercontent.com/Tech-Meet-Solutions/Bridgei2i_NLP_9th_InterIIT_Tech_Meet/main/data/dev_data_article.csv")
dft = pd.read_csv(
    "https://raw.githubusercontent.com/Tech-Meet-Solutions/Bridgei2i_NLP_9th_InterIIT_Tech_Meet/main/data/tweet_concatenated.csv", header=None)


### preprocessing

In [6]:
dft.columns = ['a', 'b', 'c', 'd', 'e']
dft3 = dft[dft['c'].str.lower().isin(companies)]['b']
dft4 = dft[dft['d'].str.lower().isin(companies)]['c']


In [7]:
df_ = pd.DataFrame()
df_['Text'] = list(dft3) + list(dft4)
df_['Mobile_Tech_Flag'] = 1
df_.head()


,Text,Mobile_Tech_Flag
0,#Samsung is now making another addition into i...,1
1,QT @AwamiWeb: Want my hands on #GalaxyS21 🥺 ; ...,1
2,QT @MadhavSheth1: We are the pioneers of 64MP ...,1
3,QT @AwamiWeb: The bestest in the smartphone in...,1
4,The Samsung #GalaxyS21 Ultra 5G is the most pr...,1


In [8]:
df_.shape


(327, 2)

In [9]:
df = pd.concat([dfa[['Text', 'Mobile_Tech_Flag']],
               df_[['Text', 'Mobile_Tech_Flag']]])
df['Mobile_Tech_Flag'].value_counts()


0.0    3025
1.0    1301
Name: Mobile_Tech_Flag, dtype: int64

In [10]:
with open('hindi_stop.txt',  encoding="utf8") as f:
    hindi_stop = list(f)
hindi_stop = [i.strip() for i in hindi_stop]

with open('hinglish_stop.txt',  encoding="utf8") as f:
    hinglish_stop = list(f)
hinglish_stop = [i.strip() for i in hinglish_stop]


In [11]:
def convert_to_lower(text):
    return text.lower()


def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))


def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english")) + hindi_stop + hinglish_stop
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)


def remove_n(text):
    single_char_pattern = r'\n'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc


def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc


In [12]:
df = df.dropna()
df['length'] = df['Text'].apply(lambda x: len(x))
df['Text'] = df['Text'].apply(lambda x: convert_to_lower(x))
df['Text'] = df['Text'].apply(lambda x: remove_n(x))
df['Text'] = df['Text'].apply(lambda x: remove_extra_white_spaces(x))
df['Text'] = df['Text'].apply(lambda x: remove_punctuation(x))
df['Text'] = df['Text'].apply(lambda x: remove_stopwords(x))
df['Text'] = df['Text'].apply(lambda x: remove_numbers(x))
df['length_after_cleaning'] = df['Text'].apply(lambda x: len(x))


In [13]:
df.reset_index()
df.head(2)


,Text,Mobile_Tech_Flag,length,length_after_cleaning
0,digitisation key buzzwords postcovid world dig...,0.0,828,603
1,increase tolerance limit cent cent real es...,0.0,4276,2721


In [14]:
df.tail(2)


,Text,Mobile_Tech_Flag,length,length_after_cleaning
325,भारत में लॉन्च xiaomi mi i है सस्ता g फोन ...,1.0,199,150
326,qt anjalisinghin phone galat choose kr haisorr...,1.0,449,309


In [15]:
vec = TfidfVectorizer()
X = vec.fit_transform(df['Text'])
X.shape


(4325, 48525)

In [16]:
X_tf = X.toarray()
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(
    X_tf, df['Mobile_Tech_Flag'].values, test_size=0.3)


### training and testing

In [19]:
automl = AutoML()
automl.fit(X_train_tf, y_train_tf, task="classification", time_budget=80)
pred = automl.predict(X_test_tf)
print(automl.model.estimator)
print(accuracy_score(y_test_tf, pred))


[flaml.automl: 12-21 16:28:13] {2599} INFO - task = classification
[flaml.automl: 12-21 16:28:13] {2601} INFO - Data split method: stratified
[flaml.automl: 12-21 16:28:13] {2604} INFO - Evaluation method: holdout
[flaml.automl: 12-21 16:28:21] {2726} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 12-21 16:28:21] {2870} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 12-21 16:28:21] {3166} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-21 16:28:22] {3296} INFO - Estimated sufficient time budget=12018s. Estimated necessary time budget=277s.
[flaml.automl: 12-21 16:28:22] {3343} INFO -  at 9.7s,	estimator lgbm's best error=0.0961,	best estimator lgbm's best error=0.0961
[flaml.automl: 12-21 16:28:22] {3166} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-21 16:28:24] {3343} INFO -  at 10.8s,	estimator lgbm's best error=0.0624,	best estimator lgbm's best error=0.0624
[flaml.automl: 

LGBMClassifier(colsample_bytree=0.7370133750309855,
               learning_rate=0.10131160192564638, max_bin=1023,
               min_child_samples=25, n_estimators=11, num_leaves=4,
               reg_alpha=0.002668211515123386, reg_lambda=0.36111742401339125,
               verbose=-1)
0.8952234206471494


In [20]:
nb = GaussianNB()
nb.fit(X_train_tf, y_train_tf)
nb_pred = nb.predict(X_test_tf)
print(accuracy_score(y_test_tf, nb_pred))


0.9214175654853621


In [21]:
svc = SVC()
svc.fit(X_train_tf, y_train_tf)
svc_pred = svc.predict(X_test_tf)
print(accuracy_score(y_test_tf, svc_pred))


0.9614791987673343


In [22]:
lr = LR()
lr.fit(X_train_tf, y_train_tf)
lr_pred = lr.predict(X_test_tf)
print(accuracy_score(y_test_tf, lr_pred))


0.9468412942989214


### handling imbalance using smote

In [23]:
smote = SMOTE(sampling_strategy=1)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tf, y_train_tf)
Counter(y_train_smote)


Counter({1.0: 2119, 0.0: 2119})

In [29]:
automl2 = AutoML()
automl2.fit(X_train_smote, y_train_smote,
            task="classification", time_budget=80)

pred = automl2.predict(X_test_tf)
print(automl2.model.estimator)
print(accuracy_score(y_test_tf, pred))


[flaml.automl: 12-21 16:42:22] {2599} INFO - task = classification
[flaml.automl: 12-21 16:42:22] {2601} INFO - Data split method: stratified
[flaml.automl: 12-21 16:42:22] {2604} INFO - Evaluation method: holdout
[flaml.automl: 12-21 16:42:25] {2726} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 12-21 16:42:25] {2870} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 12-21 16:42:25] {3166} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-21 16:42:27] {3296} INFO - Estimated sufficient time budget=18685s. Estimated necessary time budget=431s.
[flaml.automl: 12-21 16:42:27] {3343} INFO -  at 5.0s,	estimator lgbm's best error=0.0473,	best estimator lgbm's best error=0.0473
[flaml.automl: 12-21 16:42:27] {3166} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-21 16:42:29] {3343} INFO -  at 6.6s,	estimator lgbm's best error=0.0418,	best estimator lgbm's best error=0.0418
[flaml.automl: 1

: 

: 

In [25]:
nb = GaussianNB()
nb.fit(X_train_smote, y_train_smote)
nb_pred = nb.predict(X_test_tf)
print(accuracy_score(y_test_tf, nb_pred))


0.9283513097072419


In [27]:
svc = SVC()
svc.fit(X_train_smote, y_train_smote)
svc_pred = svc.predict(X_test_tf)
print(accuracy_score(y_test_tf, svc_pred))


0.9614791987673343


In [28]:

lr = LR()
lr.fit(X_train_smote, y_train_smote)
lr_pred = lr.predict(X_test_tf)
print(accuracy_score(y_test_tf, lr_pred))


0.9622496147919877
